In [ ]:
import os
import urllib.request

In [ ]:
if os.path.exists("zip.zip"):
  assert False
urllib.request.urlretrieve("https://fdc.nal.usda.gov/fdc-datasets/FoodData_Central_branded_food_csv_2021-10-28.zip", "zip.zip")
!unzip zip.zip -d data
!mkdir data/models
#urllib.request.urlretrieve("https://github.com/andreamorgar/recipe-adaptation/raw/main/models/v3/modelo3","data/models/modelo3")

AssertionError: ignored

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim
from gensim.parsing.preprocessing import preprocess_string, remove_stopwords, stem_text
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
branded_food = pd.read_csv("data/branded_food.csv")
food2 = pd.read_csv("data/food.csv")
food2 = branded_food.merge(food2, on='fdc_id')
del branded_food
food = pd.DataFrame(food2["description"])
food["branded_food_category"] = food2["branded_food_category"].astype('category')
food["brand_name"] = food2["brand_name"].fillna("").astype(str)
del food2

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
lst_stopwords = nltk.corpus.stopwords.words("english")
custom = ["gal", "oz", "t", "tsp", "teaspoon", 
          "tablespoon", "tbl", "tbs", "tbsp",
          "fl", "oz", "gil", "ounce", "ml", "l",
          "dl", "lb", "pund", "mg", "g", "kg", "gram", "cup"]
lst_stopwords.extend(custom)
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def preprocess2(name):
    name = re.sub(r'[^\w\s]', '', str(name).lower().strip())
    name = tokenizer.tokenize(name)
    name = [x for x in name if x not in lst_stopwords and len(x)>2]
    name = [x for x in name if not any(char.isdigit() for char in x)] 
    name = [lemmatizer.lemmatize(x) for x in name]
    return name

def preprocess(name):
    name = preprocess2(name)
    name = list(set(name))
    #todo bigrams
    return name

In [ ]:
lst_corpus = []
for index, row in food.iterrows():
  brand = row["brand_name"]
  x = str(row["description"])
  lst_brand = preprocess2(brand)
  lst_words = preprocess2(x)
  if not any(x in lst_words for x in lst_brand):
    lst_words = lst_brand + lst_words

  if index % 100000 == 0:
    print(index)
  #lst_grams = [" ".join(lst_words[i:i+2]) for i in range(0, len(lst_words), 1)]
  lst_corpus.append(lst_words)
bigram_mdl = Phrases(lst_corpus, min_count=5, threshold=2, delimiter=b' ')
bigrams = bigram_mdl[lst_corpus]

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000


/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
bigram_mdl.save("data/models/bigram_model.pkl")
all_sentences = list(bigrams)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
model = Word2Vec(all_sentences, min_count=3, size=300, workers=4, window=5, iter=30)
model.save("data/models/mymodel")

In [ ]:
lst_corpus[1555129]

['rocket', 'fuel', 'sauce']

In [ ]:
food

,description,branded_food_category,brand_name
0,WESSON Vegetable Oil 1 GAL,Oils Edible,
1,SWANSON BROTH BEEF,Herbs/Spices/Extracts,
2,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,Prepared Soups,
3,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,Prepared Soups,
4,SWANSON BROTH CHICKEN,Herbs/Spices/Extracts,
...,...,...,...
1555126,Manuka Hunters Bush Honey 250g,Spreads,Manuka Hunters
1555127,Manuka Hunters Bush Honey 500g,Spreads,Manuka Hunters
1555128,Manuka Hunters Bush Honey 1kg,Spreads,Manuka Hunters
1555129,Rocket Fuel Sauce 300g,Sauces,Rocket Fuel


In [ ]:
bigram_mdl["red","bull"]

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['red bull']